In [25]:
from astropy.coordinates import SkyCoord
import numpy as np
from datetime import datetime
import pandas as pd

def test_integral_query():
    from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
    
    p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
    searchquerry = SearchQuery(position=p, radius="10 degree")

    cat = IntegralQuery(searchquerry)
    f = Filter(SCW_TYPE="POINTING")
    scw_ids = cat.apply_filter(f,return_coordinates=True, remove_duplicates=False)
        
    assert scw_ids.shape == (8087, 4), "The number of pointings found is not what we expect"
    
    assert np.array_equal(scw_ids[3], np.array(['003900020030', 83.635201, 22.012722,
                                                datetime(2003, 2, 7, 7, 56, 16)], dtype=object)), \
        "A content of scw_ids is not what we expect"
        
    cat.sort_by("DEC_X")
    
    assert np.array_equal(np.array(cat.table.iloc[1114]),
                          np.array(['115900130010', 1, 'POINTING', 82.235039, 16.914862, 
                                    pd.Timestamp('2012-04-10 11:09:23'), pd.Timestamp('2012-04-10 11:44:04'), 
                                    '88602500001', 'CALIBRATION', 'PUBLIC', 'PUBLIC', 2053.0, 2038.0, 
                                    2051.0, 2053.0, 2053.0, 2053.0, 2052.0, 81747968.0, 316.022])), \
                                        "Sort_by did not produce the correct table"
                                        
    f = Filter(RA_X=Range(78., None),
           DEC_X=Range(None, 30.),
           TIME=Range(min_val='2009-11-04T00:05:23', max_val='2015-09-04T00:05:23'),
           OBS_TYPE="CALIBRATION",
           PS="PUBLIC",
           PI_NAME="PUBLIC",
           GOOD_SPI=Range(None, 500))
    
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert scw_ids.shape == (1511,), "Filter gave wrong results"
                                        
    searchquerry = SearchQuery(object_name="Cyg X-1")
    cat = IntegralQuery(searchquerry)
    f = Filter()
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert scw_ids.shape == (19151,), "Object Query gave wrong number of pointings"
    
    f = Filter(SCW_ID='004000100010',
               SCW_VER=2)
    
    scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
    
    assert scw_ids.shape == (0,), "SCW_ID or SCW_VER filter gave wrong result"
    
    
    
    
                          
                                       
test_integral_query()

In [24]:
from astropy.coordinates import SkyCoord
import numpy as np
from IntegralQuery import SearchQuery, IntegralQuery, Filter, Range
import pandas as pd

p = SkyCoord(83.6333, 22.0144, frame="icrs", unit="deg")
searchquerry = SearchQuery(position=p, radius="10 degree")
searchquerry = SearchQuery(object_name="Cyg X-1")
cat = IntegralQuery(searchquerry)
f = Filter(RA_X=Range(78., None),
           DEC_X=Range(None, 30.),
           TIME=Range(min_val='2009-11-04T00:05:23', max_val='2015-09-04T00:05:23'),
           OBS_TYPE="CALIBRATION",
           PS="PUBLIC",
           PI_NAME="PUBLIC",
           GOOD_SPI=Range(None, 500))
    
scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
cat.table, scw_ids.shape




(             SCW_ID  SCW_VER  SCW_TYPE        RA_X      DEC_X  \
 0      001130000050        1  POINTING  299.543488  35.301971   
 1      001130000061        1      SLEW  299.567740  35.250057   
 2      001130000070        1  POINTING  299.590179  35.201195   
 3      001130000080        1  POINTING  299.590179  35.201195   
 4      001130000091        1      SLEW  300.101460  36.279747   
 ...             ...      ...       ...         ...        ...   
 19161  250300240010        1  POINTING  306.328033  35.514473   
 19162  250300240021        1      SLEW  307.154813  34.567347   
 19163  250300250010        1  POINTING  307.853790  33.744278   
 19164  250300250021        1      SLEW  307.185148  33.354976   
 19165  250300260010        1  POINTING  305.735168  32.485390   
 
                START_DATE            END_DATE       OBS_ID     OBS_TYPE  \
 0     2002-11-16 20:06:11 2002-11-16 20:16:53                             
 1     2002-11-16 20:16:53 2002-11-16 20:18:23        

In [13]:
searchquerry = SearchQuery(object_name="Cyg X-1")
cat = IntegralQuery(searchquerry)
f = Filter()
scw_ids = cat.apply_filter(f,return_coordinates=False, remove_duplicates=True)
scw_ids.shape

(19151,)